# AskNews

> [AskNews](https://asknews.app) infuses any LLM with the latest news, using a single natural language query. Specifically, AskNews is enriching over 300k articles per day, indexing them into a hot vector database, and putting that vector database on an endpoint for you. When you query AskNews, you get back a prompt-optimized string that goes directly into your prompt. This means that you do not need to manage your own news RAG, and you do not need to worry about how to properly convey news information in a condensed way to your LLM.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `asknews` package itself.

```bash
pip install -U langchain-community asknews
```

We also need to set our AskNews API credentials.

In [2]:
import getpass
import os

os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

Here we show how to use the tool individually.

In [3]:
from langchain_community.tools.asknews import AskNewsSearch

tool = AskNewsSearch()

In [4]:
tool.invoke({"query": "influence of german politics on european finance"})

"\n<doc>\n[1]:\ntitle: Lindner: 'The Coarsening is Obvious'\nsummary: German Finance Minister Christian Lindner has expressed concern about the increasing aggression towards politicians and election campaigners. He blames left-wing groups for the change in atmosphere and notes that he has rarely experienced escalations in his 24 years as a politician. Lindner does not intend to change his behavior and will continue to take selfies with event attendees. He also rejects stricter penalties for attacks on politicians, saying that the current legal framework is sufficient. Experts and organizations such as the German Judges' Association and the White Ring are calling for more effective measures to combat extremism and hate speech, including prevention programs, better online disinformation education, and effective prosecution.\nsource: Berliner Morgenpost\npublished: May 11 2024 10:55\nNationality: German\nTitle: Finance Minister\nPerson: Lindner, Christian Lindner\nPolitics: politicians\nO

## Chaining
We show here how to use it as part of an [agent](/docs/modules/agents). We use the OpenAI Functions Agent, so we will need to setup and install the required dependencies for that. We will also use [LangSmith Hub](https://smith.langchain.com/hub) to pull the prompt from, so we will need to install that.

```bash
pip install -U langchain-openai langchainhub
```

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)
asknews_tool = AskNewsSearch()
tools = [asknews_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [7]:
agent_executor.invoke({"input": "influence of german politics on european finance"})



> Entering new AgentExecutor chain...

Invoking: `asknews_search` with `{'query': 'influence of German politics on European finance', 'method': 'nl'}`



<doc>
[1]:
title: The Future of Europe: A Concern for Business Leaders
summary: Business leaders are concerned that the European Union's future is not being discussed. The campaign for the European elections has focused on internal politics and geopolitics, with the war in Ukraine and the conflict between Israel and Hamas dominating the headlines. The article highlights the two different attitudes towards the EU, with some parties calling for more sovereignty and others advocating for a more integrated Europe. The author notes that no party is calling for a withdrawal from the EU, unlike in the past, and that the experience of Brexit has shown the negative consequences of leaving the EU. The article concludes that the EU needs to be defended and protected, and that business leaders need to know what the future environment will be li

{'input': 'influence of german politics on european finance',
 'output': "German politics, particularly the positions of different German parties, have a significant influence on European finance. The main German parties have differing stances on issues such as climate change, migration, and European integration. For example, the Greens aim to transform the EU into a federal state, the Social Democrats prioritize social solidarity and a strong European social security system, the Free Democrats advocate for individual responsibility and cost-cutting measures, the Christian Democrats focus on balancing interests and a strong defense, the Alternative for Germany (AfD) wants to abolish the EU and establish a new union of European nations, and the Left Party seeks radical reforms and a more equal society.\n\nAdditionally, German Finance Minister Christian Lindner has been pressing for spending cuts in the 2025 budget, questioning the effectiveness of projects by Foreign Minister Annalena B